<a href="https://colab.research.google.com/github/tusharsharm/MDSC-302/blob/main/24040208008-MDSC-302(P)-V.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#24040208008-MDSC-302(P)-V

In [ ]:

from torchvision import datasets, transforms
data_path = '/content/p1ch7/'



In [ ]:
# Load the CIFAR10 training dataset
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True, # Specify data path, set to training set, and download if not present
    transform=transforms.Compose([
        transforms.ToTensor(), # Convert images to PyTorch tensors
        transforms.Normalize((0.4915, 0.4823, 0.4468), # Normalize the image tensors with mean and standard deviation
                             (0.2470, 0.2435, 0.2616))
    ]))

100%|██████████| 170M/170M [00:03<00:00, 43.6MB/s]


In [ ]:
# Load the CIFAR10 validation dataset
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False, # Specify data path, set to validation set, and do not download as it should be present
    transform=transforms.Compose([
        transforms.ToTensor(), # Convert images to PyTorch tensors
        transforms.Normalize((0.4915, 0.4823, 0.4468), # Normalize the image tensors with mean and standard deviation
                             (0.2470, 0.2435, 0.2616))
    ]))

In [ ]:
# Define a mapping for the original labels (0 and 2) to new labels (0 and 1)
label_map = {0: 0, 2: 1}
# Define the names for the new classes
class_names = ['airplane', 'bird']

# Create a new dataset 'cifar2' by filtering the training data (cifar10)
# to include only images with original labels 0 (airplane) or 2 (bird)
# The original labels are mapped to the new labels using label_map
cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]

# Create a new dataset 'cifar2_val' by filtering the validation data (cifar10_val)
# to include only images with original labels 0 (airplane) or 2 (bird)
# The original labels are mapped to the new labels using label_map
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Create a DataLoader for the training dataset
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=True)

# Define the neural network model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # First convolutional layer: 3 input channels (RGB), 16 output channels, 3x3 kernel, 1 pixel padding
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        # Second convolutional layer: 16 input channels, 8 output channels, 3x3 kernel, 1 pixel padding
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        # First fully connected layer: input size 8*8*8 (from pooled conv output), output size 32
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        # Second fully connected layer: input size 32, output size 2 (for two classes: airplane or bird)
        self.fc2 = nn.Linear(32, 2)

    # Define the forward pass
    def forward(self, x):
        # Apply first convolution, ReLU activation, and 2x2 max pooling
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        # Apply second convolution, ReLU activation, and 2x2 max pooling
        out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
        # Flatten the output for the fully connected layers
        out = out.view(-1, 8 * 8 * 8)
        # Apply first fully connected layer and tanh activation
        out = torch.tanh(self.fc1(out))
        # Apply second fully connected layer (output layer)
        out = self.fc2(out)
        return out

# Instantiate the network model
model = Net()

# Define the learning rate
learning_rate = 1e-2

# Define the optimizer (Stochastic Gradient Descent)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Define the loss function (Cross-Entropy Loss for classification)
loss_fn = nn.CrossEntropyLoss()

# Define the number of training epochs
n_epochs = 100

# Training loop
for epoch in range(n_epochs):
    # Iterate over batches in the training data
    for imgs, labels in train_loader:
        # Get model outputs for the current batch
        outputs = model(imgs)
        # Calculate the loss
        loss = loss_fn(outputs, labels)

        # Zero the gradients, perform backpropagation, and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the loss after each epoch
    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

/tmp/ipython-input-1016970036.py:67: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:835.)
  print("Epoch: %d, Loss: %f" % (epoch, float(loss)))


Epoch: 0, Loss: 0.543850
Epoch: 1, Loss: 0.580809
Epoch: 2, Loss: 0.398413
Epoch: 3, Loss: 0.653744
Epoch: 4, Loss: 0.241510
Epoch: 5, Loss: 0.543811
Epoch: 6, Loss: 0.299707
Epoch: 7, Loss: 0.454672
Epoch: 8, Loss: 0.205742
Epoch: 9, Loss: 0.565841
Epoch: 10, Loss: 0.571185
Epoch: 11, Loss: 0.454910
Epoch: 12, Loss: 0.391203
Epoch: 13, Loss: 0.308612
Epoch: 14, Loss: 0.222387
Epoch: 15, Loss: 0.162261
Epoch: 16, Loss: 0.537388
Epoch: 17, Loss: 0.440321
Epoch: 18, Loss: 0.216916
Epoch: 19, Loss: 0.211631
Epoch: 20, Loss: 0.468714
Epoch: 21, Loss: 0.509593
Epoch: 22, Loss: 0.434849
Epoch: 23, Loss: 0.253481
Epoch: 24, Loss: 0.102826
Epoch: 25, Loss: 0.407321
Epoch: 26, Loss: 0.315105
Epoch: 27, Loss: 0.088072
Epoch: 28, Loss: 0.529045
Epoch: 29, Loss: 0.147682
Epoch: 30, Loss: 0.310716
Epoch: 31, Loss: 0.316833
Epoch: 32, Loss: 0.154504
Epoch: 33, Loss: 0.459508
Epoch: 34, Loss: 0.165526
Epoch: 35, Loss: 0.327768
Epoch: 36, Loss: 0.543515
Epoch: 37, Loss: 0.181486
Epoch: 38, Loss: 0.259

In [ ]:
# Create a DataLoader for the training dataset, ensuring shuffle is False for consistent evaluation
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=False)

correct = 0
total = 0

# Disable gradient calculation for evaluation
with torch.no_grad():
    # Iterate over the training data
    for imgs, labels in train_loader:
        # Get model predictions. Pass the images directly to the model.
        outputs = model(imgs)
        # Get the predicted class by finding the index of the maximum output value
        _, predicted = torch.max(outputs, dim=1)
        # Update the total number of samples
        total += labels.shape[0]
        # Update the number of correct predictions
        correct += int((predicted == labels).sum())

# Print the accuracy on the training set
print("Accuracy: %f" % (correct / total))

Accuracy: 0.949300


In [ ]:
# Create a DataLoader for the validation dataset, ensuring shuffle is False for consistent evaluation
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,
                                         shuffle=False)

correct = 0
total = 0

# Disable gradient calculation for evaluation
with torch.no_grad():
    # Iterate over the validation data
    for imgs, labels in val_loader:
        # Get model predictions. Pass the images directly to the model.
        outputs = model(imgs)
        # Get the predicted class by finding the index of the maximum output value
        _, predicted = torch.max(outputs, dim=1)
        # Update the total number of samples
        total += labels.shape[0]
        # Update the number of correct predictions
        correct += int((predicted == labels).sum())

# Print the accuracy on the validation set
print("Accuracy: %f" % (correct / total))

Accuracy: 0.890500
